In [1]:
from pathlib import Path
import glob
import requests
import pandas as pd

In [9]:
api_key: str = "********************"

In [16]:
# writing out the start and end dates out here as-is because why not (gpt generated looking ahh)
start_dates: list[str] = ['2023-10-01', '2023-10-05', '2023-10-09', '2023-10-13', '2023-10-17', '2023-10-21', '2023-10-25', '2023-10-29', '2023-11-01', '2023-11-05', '2023-11-09', '2023-11-13', '2023-11-17', '2023-11-21', '2023-11-25', '2023-11-29', '2023-12-01', '2023-12-05', '2023-12-09', '2023-12-13', '2023-12-17', '2023-12-21', '2023-12-25', '2023-12-29', '2024-01-01', '2024-01-05', '2024-01-09', '2024-01-13', '2024-01-17', '2024-01-21', '2024-01-25', '2024-01-29']
end_dates: list[str] = ['2023-10-04', '2023-10-08', '2023-10-12', '2023-10-16', '2023-10-20', '2023-10-24', '2023-10-28', '2023-10-31', '2023-11-04', '2023-11-08', '2023-11-12', '2023-11-16', '2023-11-20', '2023-11-24', '2023-11-28', '2023-11-30', '2023-12-04', '2023-12-08', '2023-12-12', '2023-12-16', '2023-12-20', '2023-12-24', '2023-12-28', '2023-12-31', '2024-01-04', '2024-01-08', '2024-01-12', '2024-01-16', '2024-01-20', '2024-01-24', '2024-01-28', '2024-01-31']

In [17]:
for date_pair in zip(start_dates, end_dates):

    base_url: str = f"https://www.goflightlabs.com/historical/{date_pair[0]}"

    query_params: dict = {
        "access_key" : api_key,
        "code": "YUL",
        "type": "departure",
        "date_to": date_pair[1]
    }

    response = requests.get(base_url, params = query_params)

    if response.status_code == 200:

        print("Pull successful - writing....")

        response_json = response.json()

        required_data = response_json.get('data', [])

        structured_data: list = []

        for flight in required_data:

            if isinstance(flight, dict):
                structured_data.append({
                    'Type': flight['type'],
                    'Status': flight['status'],
                    'Departure IATA Code': flight['departure'].get('iataCode', ''),
                    'Departure Gate': flight['departure'].get('gate', ''),
                    'Departure Delay (min)': flight['departure'].get('delay', 0),
                    'Scheduled Departure Time': flight['departure'].get('scheduledTime', ''),
                    'Estimated Departure Time': flight['departure'].get('estimatedTime', ''),
                    'Actual Departure Time': flight['departure'].get('actualTime', ''),
                    'Arrival IATA Code': flight['arrival'].get('iataCode', ''),
                    'Arrival Terminal': flight['arrival'].get('terminal', ''),
                    'Scheduled Arrival Time': flight['arrival'].get('scheduledTime', ''),
                    'Estimated Arrival Time': flight['arrival'].get('estimatedTime', ''),
                    'Airline Name': flight['airline'].get('name', ''),
                    'Flight Number': flight['flight'].get('number', ''),
                    'IATA Flight Number': flight['flight'].get('iataNumber', '')
                })
        
        flight_info_df: pd.DataFrame = pd.DataFrame(structured_data)

        flight_info_df = flight_info_df.drop_duplicates(subset = ['Type', 'Status', 'Departure IATA Code', 'Departure Gate', 'Departure Delay (min)', 'Scheduled Departure Time', 'Estimated Departure Time', 'Actual Departure Time', 'Arrival IATA Code', 'Arrival Terminal'])

        flight_info_df.to_csv(f"./flight_data/YUL_Departures_{date_pair[0]}_to_{date_pair[1]}.csv", index = False)

        print("File written - onto the next one")
    

Pull successful - writing....
File written - onto the next one
Pull successful - writing....
File written - onto the next one
Pull successful - writing....
File written - onto the next one
Pull successful - writing....
File written - onto the next one
Pull successful - writing....
File written - onto the next one
Pull successful - writing....
File written - onto the next one
Pull successful - writing....
File written - onto the next one
Pull successful - writing....
File written - onto the next one
Pull successful - writing....
File written - onto the next one
Pull successful - writing....
File written - onto the next one
Pull successful - writing....
File written - onto the next one
Pull successful - writing....
File written - onto the next one
Pull successful - writing....
File written - onto the next one
Pull successful - writing....
File written - onto the next one
Pull successful - writing....
File written - onto the next one
Pull successful - writing....
File written - onto the n

In [23]:
# Concatenating all of the output CSV files from the API pull into one big DataFrame

flight_df_3mo = pd.DataFrame()

for csv_file in glob.glob("./flight_data/*.csv"):

    try:

        file: pd.DataFrame = pd.read_csv(csv_file)

        if not file.empty:

            flight_df_3mo = pd.concat([flight_df_3mo, file], ignore_index = True)

    except pd.errors.EmptyDataError:

        print(f"Skipping empty file: {csv_file}")

Skipping empty file: ./flight_data/YUL_Departures_2024-01-25_to_2024-01-28.csv


In [24]:
flight_df_3mo

,Type,Status,Departure IATA Code,Departure Gate,Departure Delay (min),Scheduled Departure Time,Estimated Departure Time,Actual Departure Time,Arrival IATA Code,Arrival Terminal,Scheduled Arrival Time,Estimated Arrival Time,Airline Name,Flight Number,IATA Flight Number
0,departure,active,yul,a61,23,2024-01-12t19:00:00.000,2024-01-12t19:00:00.000,2024-01-12t19:22:00.000,lhr,2,2024-01-13t06:30:00.000,2024-01-13t06:11:00.000,aegean airlines,3089,a33089
1,departure,active,yul,a63,33,2024-01-12t19:05:00.000,2024-01-12t19:29:00.000,2024-01-12t19:37:00.000,cdg,2e,2024-01-13t08:05:00.000,2024-01-13t07:45:00.000,delta air lines,8612,dl8612
2,departure,active,yul,a57,0,2024-01-12t19:05:00.000,2024-01-12t19:05:00.000,NaN,ams,NaN,2024-01-13t08:05:00.000,NaN,air france,8276,af8276
3,departure,active,yul,a49,115,2024-01-12t19:10:00.000,2024-01-12t21:05:00.000,NaN,yyz,1,2024-01-12t20:46:00.000,NaN,azul,7807,ad7807
4,departure,active,yul,a6,30,2024-01-12t19:25:00.000,2024-01-12t19:55:00.000,NaN,yyz,3,2024-01-12t21:00:00.000,NaN,porter,130,p3130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30820,departure,active,yul,a65,57,2024-01-19t22:15:00.000,2024-01-19t23:12:00.000,NaN,mbj,NaN,2024-01-20t06:40:00.000,NaN,air transat,150,ts150
30821,departure,active,yul,NaN,90,2024-01-20t12:30:00.000,2024-01-20t14:00:00.000,NaN,yyz,NaN,2024-01-20t13:33:00.000,NaN,air canada,7053,ac7053
30822,departure,active,yul,NaN,0,2024-01-20t15:45:00.000,2024-01-20t15:45:00.000,2024-01-20t15:35:00.000,ysj,NaN,2024-01-20t18:04:00.000,2024-01-20t17:58:00.000,private owner,23,NaN
30823,departure,active,yul,NaN,41,2024-01-20t15:00:00.000,2024-01-20t15:41:00.000,NaN,yqm,NaN,2024-01-20t17:20:00.000,NaN,air canada,7125,ac7125
